<a href="https://colab.research.google.com/github/evinracher/3008410-intelligent-systems/blob/main/week2/exercise1/Translator_with_BLUE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%env TF_USE_LEGACY_KERAS=1

!pip -q uninstall -y keras transformers tensorflow tf-keras safetensors
!pip -q install \
  "tensorflow==2.20.0" \
  "tf-keras" \
  "transformers==4.49.0" \
  "safetensors==0.4.5" \
  "datasets==2.20.0" \
  "evaluate==0.4.2" \
  "accelerate==0.34.2" \
  "sentencepiece==0.2.0"


env: TF_USE_LEGACY_KERAS=1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-hub 0.21.1 requires tensorflow-text; platform_system != "Windows", which is not installed.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.


In [ ]:

from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM, AdamWeightDecay, pipeline
from datasets import Dataset
import tensorflow as tf
import evaluate
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# --- Load dataset ---
folder_path = "/content"
dataset_name = "eng_small.csv"
path = os.path.join(folder_path, dataset_name)
data = Dataset.from_csv(path, encoding='utf-8')
data = data.train_test_split(test_size=0.3)

# --- Tokenizer & Model ---
tokenizer = AutoTokenizer.from_pretrained("t5-small") # t5-small model
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

# --- Preprocessing ---
prefix = "translate: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = data.map(preprocess_function, batched=True, remove_columns=["engl", "spa"])

# --- Data Collator ---
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

# --- Prepare TF datasets ---
tf_train_set = model.prepare_tf_dataset(tokenized_data["train"], shuffle=True, batch_size=16, collate_fn=data_collator)
tf_test_set = model.prepare_tf_dataset(tokenized_data["test"], shuffle=False, batch_size=16, collate_fn=data_collator)

# --- Optimizer ---
optimizer = AdamWeightDecay(learning_rate=2e-4, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)  # Use internal loss handling

# --- BLEU Callback ---
bleu = evaluate.load("bleu")
class BLEUCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, tokenizer, raw_dataset):
        self.model = model
        self.tokenizer = tokenizer
        self.raw_dataset = raw_dataset

    def on_epoch_end(self, epoch, logs=None):
        predictions = []
        references = []
        for sample in self.raw_dataset:
            input_text = prefix + sample["engl"]
            reference = sample["spa"]
            inputs = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)
            output = model.generate(**inputs, max_length=128)
            pred = tokenizer.decode(output[0], skip_special_tokens=True)
            predictions.append(pred)
            references.append([reference])
        result = bleu.compute(predictions=predictions, references=references)
        print(f"\n📘 Epoch {epoch+1} - BLEU Score: {round(result['bleu'], 4)}")

# --- Train Model ---
epochs = 5
raw_test_data = data["test"].to_pandas().to_dict(orient='records')
history = model.fit(tf_train_set, validation_data=tf_test_set, epochs=epochs,
                    callbacks=[BLEUCallback(model, tokenizer, raw_test_data)])

# --- Save model ---
model_path = os.path.join('model', f"NMT-epocs-{epochs}")
model.save_pretrained(model_path)

# --- Export Predictions ---
print("Exporting predictions...")
predictions, references, sources = [], [], []
for sample in raw_test_data:
    input_text = prefix + sample["engl"]
    reference = sample["spa"]
    inputs = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)
    output = model.generate(**inputs, max_length=128)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(pred)
    references.append(reference)
    sources.append(sample["engl"])

df = pd.DataFrame({
    "Source (EN)": sources,
    "Reference (ES)": references,
    "Prediction": predictions
})
csv_path = os.path.join(folder_path, f"predictions_epoch{epochs}.csv")
df.to_csv(csv_path, index=False)
print(f"Predictions saved to: {csv_path}")

# --- Optional: plot BLEU score summary ---
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])["bleu"]
plt.figure(figsize=(6, 4))
plt.bar(["BLEU Score"], [bleu_score], color='skyblue')
plt.title("BLEU Score After Training")
plt.ylim(0, 1)
plt.ylabel("Score")
plt.tight_layout()
plt.show()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/14074 [00:00<?, ? examples/s]

Map:   0%|          | 0/6033 [00:00<?, ? examples/s]

Epoch 1/5
878/879 [============================>.] - ETA: 0s - loss: 2.2613

## Exercises

- Try with max_length=512 and report
- Double the epochs = 10 and report
- Reverse ["spa"] to ["eng"], which one is easier to translate by BLEU metric?
- Move learning_rate=2e-4 what happened?
- What if batch_size=32?
- What does it mean if prefix = "summarize: "?
  